In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
data_path_file='DelayedFlights.csv'
df =pd.read_csv(data_path_file)

Centraremos el analisis en las caracteristicas de interés: features y lo guardaremos en el dataframe X

In [3]:
features=[ 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'ActualElapsedTime',
          'CRSElapsedTime', 'AirTime', 'ArrDelay','DepDelay', 'Distance', 'Cancelled', 'Diverted', 'UniqueCarrier']

In [4]:
X= df[features]

In [5]:
X.shape

(1936758, 14)

Dado:

* la variablidad en los retrasos por compañias 
* el gran numero de datos disponibles
* Incluir la variable categorica 'UniqueCarrier' con 20 valores diferentes con un One-Hot Encoding seria ineficiente.

Escogeremos una compañia para realizar nuestro modelo.

Escogemos US Airways 'US' lo cual constituye aproximadamente el 5% de los vuelos.

In [6]:
X = X[ X['UniqueCarrier'] == 'US' ]

In [7]:
X.shape

(98425, 14)

Supongamos que **queremos entrenar un modelo que sea capaz de estimar la caracteristica 'ArrDelay' una vez el vuelo ya ha salido pero obviamente aun no ha llegado**. 

Para ello 'Arrdelay' no ha de tener faltantes. tratandose de un numero relativamente bajo descartaremos las filas con 'ArrDelay' faltantes para toda la matriz de caracteristicas X

In [8]:
nodelay = ~(X['ArrDelay'].isnull())
X= X[nodelay]

In [9]:
X.shape

(98007, 14)

Guardamos en la variable y la caracteristica 'ArrDelay' y la eliminaremos de la matriz de caracteristicas X

In [10]:
y= X.ArrDelay
X.drop( columns='ArrDelay', inplace= True )

In [11]:
X.shape

(98007, 13)

Por tanto las caracteristicas de la matrix X seran las contenidas en la lista feat_X. Por ser nulas Cancelled y Diverted se eliminan tambien. **Como el vuelo no ha llegado en el momento de estimar su retraso, no dispondremos de 'ArrTime', 'ActualElapsedTime' ni 'Airtime'**

In [12]:
feat_X=[ 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime',
          'CRSElapsedTime','DepDelay', 'Distance']

In [13]:
X= X[feat_X]

In [14]:
X.shape

(98007, 7)

Para los modelos Lineales y RandomForest no es necesario standardizar los datos. No obstante **para el modelo MLPRegressor si es necesario para mejorar la convergencia.** Asi pues **standardizaremos los datos de la matriz X**

Como se standardiza en base a los datos de train como paso previo separamos con train_test_split()

In [15]:
# Dividimos los datos (X,y) en training y test (X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(78405, 7) (19602, 7) (78405,) (19602,)


Standardizamos ambos dataframes: X_train, X_test

In [16]:
# Instancia del scaler
scaler = preprocessing.StandardScaler()

In [17]:
# Fit to TRAIN data. Then transform it.
M_train = scaler.fit_transform(X_train)
#Volvemos a meter la matriz en dataframe
X_train = pd.DataFrame(M_train, columns= feat_X )

# Transform TEST data 
M_test = scaler.transform(X_test)
#Volvemos a meter la matriz en dataframe
X_test= pd.DataFrame(M_test, columns= feat_X )

In [18]:
print(X_train.shape, X_test.shape)

(78405, 7) (19602, 7)


--------------------------
## Ejercicio 1


Crea almenos 3 modelos de Regresion diferentes para intentar predecir el retraso de los vuelos 'ArrDelay'

-------------------------


Usaremos los siguientes modelos:

* Modelo Lineal
* Random Forest
* MLP Regressor

Generaremos instancias de los modelos, los entrenaremos y realizaremos las predicciones para todos ellos con los parametros por defecto.

In [19]:
# Modelo Lineal
linear_1 = LinearRegression()

#Entrenamos el modelo lineal
linear_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_lin1= linear_1.predict(X_test)

In [20]:
# Modelo Random Forest
forest_1 = RandomForestRegressor()

#Entrenamos el Forest Regressor
forest_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_for1 = forest_1.predict(X_test)

In [21]:
# Modelo MLPRegressor
mlp_1 = MLPRegressor()

#Entrenamos el MLP Regressor
mlp_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_mlp1 = mlp_1.predict(X_test)

--------------------------
## Ejercicio 2

Compara los modelos segun el MSE y el R2.

--------------------------

Evaluaremos la calidad del modelo con los datos de test

In [22]:
r2_lin1 = linear_1.score(X_test, y_test)
mse_lin1 = mean_squared_error(y_test, y_pred_lin1)

r2_for1 = forest_1.score(X_test, y_test)
mse_for1 = mean_squared_error(y_test, y_pred_for1)

r2_mlp1= mlp_1.score(X_test, y_test)
mse_mlp1 = mean_squared_error(y_test, y_pred_mlp1)

In [23]:
print('Linear Regression | R^2: ', round(r2_lin1,6), ' | ', 'MSE: ', round(mse_lin1,6) )
print('Random Forest     | R^2: ', round(r2_for1,6), ' | ', 'MSE: ', round(mse_for1,6) )
print('MLPRegressor      | R^2: ', round(r2_mlp1,6), ' | ', 'MSE: ', round(mse_mlp1,6) )

Linear Regression | R^2:  0.863595  |  MSE:  354.007001
Random Forest     | R^2:  0.856217  |  MSE:  373.155913
MLPRegressor      | R^2:  0.866151  |  MSE:  347.374154


In [24]:
linear_1.coef_

array([-3.06781027e-02,  9.95221972e-01, -1.59531481e+00,  8.64666690e-01,
        1.13484463e+00,  4.63947048e+01, -3.17960071e+00])

El orden de los modelos de mejor a peor serian el MLPRegressor, Regresion Lineal y por ultimo Random Forest. Orden creciente de R^2 y decreciente de MSE.

---------------------------

## Ejercicio 3


Utiliza los diferentes parametros que admiten

----------------------------------------------

Para el modelo lineal implementaremos la variante Ridge que incluye en la funcion a minimizar la suma de los coeficientes al cuadrado.

Ridge Linear Regression:

* alpha: factor sobre la suma de coeficientes del modelo al cuadrado. alpha=0 seria el modelo lineal standard.

Random Forest:
* n_estimators: numero de arboles del modelo. Def:100
* max_depth: Maximos niveles de division del modelo. Def:None. Hace split hasta que todas las leaf sean puras o con samples<min_samples_split
* min_samples_split: Numero minimo de muestras que ha de tener el nodo para poder hacer split. Def:2 
* min_samples_leaf: Solo se hará split del nodo si deja como minimo min_samples_leaf en ambas ramas. Def:1
* criterion: El criterio de optimizacion. Por defecto es el squared error. Def:'squared_error'
* n_jobs: Divide la computacion del modelo en los nucleos del procesador del ordenador local, 4 en mi caso. Def:None means 1
* verbose: int. Controla el feddback al entrenar y predecir. default=0

MLP Regressor:
* hidden_layer_sizes: Tamaño de las capas ocultas. Se da en tupla. La red ademas tendra una capa incial y la final de salida. Def:(100,)
* activation: Funcion de activacion. Def:'relu'
* solver: El solver tiene 3 opciones la Def:'adam'
* learning_rate: fija el tipo de actualizacion de los pesos. default='constant'.
* learning_rate_init: El valor inicial del learning rate para actualizar los weights. default= 0.001
* tol: Cuando loss o score no mejora por lo menos tol durante 'n_iter_no_change' iteraciones consecutivas para el training(a no ser que learning_rate='adaptive') Defecto 1E-4.  
* max_iter: Si no llega a la convergencia para en la iteracion marcada por este parametro. Default:200
* n_iter_no_change: Maximo numero de epochs en las que no se consigue una mejora de 'tol'. Default:10
* verbose:int. Muestra la informacion del training. Def:0

In [25]:
linear_2 = Ridge(alpha=1, random_state=33)

#Entrenamos el modelo lineal
linear_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_lin2 = linear_2.predict(X_test)

In [26]:
# Modelo Random Forest
forest_2 = RandomForestRegressor(n_estimators = 200, 
                               max_depth=8, min_samples_split= 50, min_samples_leaf=20,
                               random_state = 33, criterion= 'squared_error', n_jobs=4,
                                verbose=0)

#Entrenamos el Forest Regressor
forest_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_for2 = forest_2.predict(X_test)

In [69]:
# Modelo MLPRegressor
mlp_2 = MLPRegressor(hidden_layer_sizes= (140,20), activation='relu', solver='adam',
                      learning_rate_init= 0.005, learning_rate='constant', max_iter=200,
                      verbose=2, n_iter_no_change=10, tol=0.01, random_state=33)

#Entrenamos el MLP Regressor
mlp_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_mlp2 = mlp_2.predict(X_test)

Iteration 1, loss = 330.27287621
Iteration 2, loss = 174.48419170
Iteration 3, loss = 173.77435812
Iteration 4, loss = 173.45426210
Iteration 5, loss = 172.89292245
Iteration 6, loss = 172.72522488
Iteration 7, loss = 172.82583959
Iteration 8, loss = 172.64346624
Iteration 9, loss = 172.96717558
Iteration 10, loss = 172.68139183
Iteration 11, loss = 172.08906230
Iteration 12, loss = 171.98396393
Iteration 13, loss = 171.89220632
Iteration 14, loss = 171.89937866
Iteration 15, loss = 171.66694029
Iteration 16, loss = 172.00945894
Iteration 17, loss = 171.34450158
Iteration 18, loss = 171.59752676
Iteration 19, loss = 171.50347744
Iteration 20, loss = 171.35942744
Iteration 21, loss = 171.10934088
Iteration 22, loss = 171.49879327
Iteration 23, loss = 170.94434586
Iteration 24, loss = 171.13240620
Iteration 25, loss = 171.58115997
Iteration 26, loss = 171.12105403
Iteration 27, loss = 170.84384335
Iteration 28, loss = 170.69853920
Iteration 29, loss = 170.61053422
Iteration 30, loss = 17

In [70]:
r2_lin2 = linear_2.score(X_test, y_test)
mse_lin2 = mean_squared_error(y_test, y_pred_lin2)

r2_for2 = forest_2.score(X_test, y_test)
mse_for2 = mean_squared_error(y_test, y_pred_for2)

r2_mlp2= mlp_2.score(X_test, y_test)
mse_mlp2 = mean_squared_error(y_test, y_pred_mlp2)

In [71]:
print('Linear Regression | R^2: ', round(r2_lin1,6), ' | ', 'MSE: ', round(mse_lin1,6) )
print('Ridge Linear Reg  | R^2: ', round(r2_lin2,6), ' | ', 'MSE: ', round(mse_lin2,6) )
print('Random Forest 1   | R^2: ', round(r2_for1,6), ' | ', 'MSE: ', round(mse_for1,6) )
print('Random Forest 2   | R^2: ', round(r2_for2,6), ' | ', 'MSE: ', round(mse_for2,6) )
print('MLPRegressor  1   | R^2: ', round(r2_mlp1,6), ' | ', 'MSE: ', round(mse_mlp1,6) )
print('MLPRegressor  2   | R^2: ', round(r2_mlp2,6), ' | ', 'MSE: ', round(mse_mlp2,6) )

Linear Regression | R^2:  0.863595  |  MSE:  354.007001
Ridge Linear Reg  | R^2:  0.863595  |  MSE:  354.006993
Random Forest 1   | R^2:  0.856217  |  MSE:  373.155913
Random Forest 2   | R^2:  0.864226  |  MSE:  352.368567
MLPRegressor  1   | R^2:  0.866151  |  MSE:  347.374154
MLPRegressor  2   | R^2:  0.866411  |  MSE:  346.698824


--------------------

## Ejercicio 4

Compara el rendimiento entre usar o no usar el train/test

----------------------

In [30]:
#Standardizamos todo el database
scaler2 = preprocessing.StandardScaler()
# Fit to ALL data. Then transform it.
M = scaler2.fit_transform(X)
#Volvemos a meter la matriz en el dataframe XX
XX = pd.DataFrame(M, columns= feat_X )

In [31]:
XX.shape

(98007, 7)

In [32]:
# Modelo Lineal
model1 = LinearRegression(n_jobs=4)
model1.fit(XX, y)
y_pred_1= model1.predict(XX)

# Modelo Random Forest
model2 = RandomForestRegressor()
model2.fit(XX, y)
y_pred_2= model2.predict(XX)

# Modelo MLPRegressor
model3 = MLPRegressor()
model3.fit(XX, y)
y_pred_3= model3.predict(XX)

c:\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Evaluamos las predicciones

In [33]:
r2_m1= model1.score(XX, y)
mse1 = mean_squared_error(y, y_pred_1)
r2_m2= model2.score(XX, y)
mse2 = mean_squared_error(y, y_pred_2)
r2_m3= model3.score(XX, y)
mse3 = mean_squared_error(y, y_pred_3)

print('Linear Reg. All Data | R^2: ', round(r2_m1,6), '  | ', 'MSE:', round(mse1,6) )
print('Linear Reg.  Test D  | R^2: ', round(r2_lin1,6), ' | ', 'MSE: ', round(mse_lin1,6) )
print('Forest Reg. All Data | R^2: ', round(r2_m2,6), ' | ', 'MSE:', round(mse2,6) )
print('Forest Reg.  Test D  | R^2: ', round(r2_for1,6), ' | ', 'MSE: ', round(mse_for1,6) )
print('MLP Reg.    All Data | R^2: ', round(r2_m3,6), ' | ', 'MSE:', round(mse3,6) )
print('MLP Reg.    Test D   | R^2: ', round(r2_mlp1,6), ' | ', 'MSE: ', round(mse_mlp1,6) )

Linear Reg. All Data | R^2:  0.861602   |  MSE: 349.812151
Ridge Linear Test D  | R^2:  0.863595  |  MSE:  354.007001
Forest Reg. All Data | R^2:  0.977595  |  MSE: 56.629982
Random For2  Test D  | R^2:  0.856217  |  MSE:  373.155913
MLP Reg.  All Data   | R^2:  0.865594  |  MSE: 339.720798
MLPRegressor  2      | R^2:  0.866151  |  MSE:  347.374154


En general hacer la validacion con los mismos datos con los que se entrena el modelo dara R^2 mayor y MSE menor. Esto indicaria que las predicciones son mejores. No obstante es mejor hacer un test del modelo con datos que no hayan intervenido en la construccion del modelo. 

Asi se puede valorar la prediccion de datos que el modelo no ha usado. El sobre-entreno o 'Over Fitting' puede llevar a un modelo que prediga con precision los datos con los que fue construido pero le resulte dificil realizar una buena prediccion de datos que aun no ha visto.

-----------------------

## Ejercicio 5

Realiza algun proceso de Ingenieria de Variables para mejorar la prediccion.

-----------------------

Añadiremos la mejor variable predictora 'DepDelay' al cuadrado y al cubo en la matriz de caracteristicas. 

In [34]:
X_fe = X

In [35]:
feat_X_fe = feat_X
feat_X_fe.append('DD2')
feat_X_fe.append('DD3')
feat_X_fe

['DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'CRSArrTime',
 'CRSElapsedTime',
 'DepDelay',
 'Distance',
 'DD2',
 'DD3']

In [36]:
X_fe['DD2']=X_fe['DepDelay']**2

In [37]:
X_fe['DD3']=X_fe['DepDelay']**3

In [38]:
X_fe.head()

,DayOfWeek,DepTime,CRSDepTime,CRSArrTime,CRSElapsedTime,DepDelay,Distance,DD2,DD3
81557,2,1925.0,1855,2114,139.0,30.0,646,900.0,27000.0
81558,2,152.0,146,917,331.0,6.0,2551,36.0,216.0
81559,2,1407.0,1400,1514,74.0,7.0,227,49.0,343.0
81560,2,2206.0,2142,2300,258.0,24.0,1747,576.0,13824.0
81561,2,1453.0,1440,1703,143.0,13.0,665,169.0,2197.0


In [39]:
# Dividimos los datos (X,y) en training y test (X_train, X_test, y_train, y_test)

X_fe_train, X_fe_test, y_train, y_test = train_test_split(X_fe, y, train_size=0.8, test_size=0.2, random_state=33)
print(X_fe_train.shape, X_fe_test.shape, y_train.shape, y_test.shape)

(78405, 9) (19602, 9) (78405,) (19602,)


In [40]:
scaler3 = preprocessing.StandardScaler()

In [41]:
# Fit to TRAIN data. Then transform it.
M_fe_train = scaler3.fit_transform(X_fe_train)
#Volvemos a meter la matriz en dataframe
X_fe_train = pd.DataFrame(M_fe_train, columns= feat_X_fe )

# Transform TEST data 
M_fe_test = scaler3.transform(X_fe_test)
#Volvemos a meter la matriz en dataframe
X_fe_test= pd.DataFrame(M_fe_test, columns= feat_X_fe )

In [42]:
# Modelo Lineal
model1b = LinearRegression(n_jobs=4)

#Entrenamos el modelo lineal
model1b.fit(X_fe_train, y_train)

y_pred_1b= model1b.predict(X_fe_test)

In [43]:
r2_m1b= model1b.score(X_fe_test, y_test)
mse1b = mean_squared_error(y_test, y_pred_1b)

print('Linear Regression X_fe Validation Results: R^2:', round(r2_m1b,6), 'MSE:', round(mse1b,6) )

Linear Regression X_fe Validation Results: R^2: 0.863624 MSE: 353.930786


Observamos como la introduccion de 'DepDelay' al cuadrado y al cubo ha mejorado levemente la prediccion del modelo. El modelo lineal sin estas variables habia dado R^2:0.863595  y MSE:354.007001 Observamos un valor de R^2 mayor y menor MSE para las predicciones realizadas sobre el conjunto de datos de test.



----------------------------

## Ejercicio 6

No utilices la variable 'DepDelay' en las prediciones

----------------------------

Definamos la lista de variables a usar X_train_red y X_test_red 

In [44]:
X_train_red = X_train.drop( columns='DepDelay', inplace= False )
X_test_red = X_test.drop(columns='DepDelay', inplace=False )

In [45]:
# Modelo Lineal
model1c = LinearRegression(n_jobs=4)

# Modelo Random Forest
model2c = RandomForestRegressor(n_estimators = 200, 
                               max_depth=10, min_samples_split= 50, min_samples_leaf=20,
                               random_state = 33, criterion= 'squared_error', n_jobs=4)

# Modelo MLPRegressor
model3c = MLPRegressor(hidden_layer_sizes= (20,10,), activation='relu', solver='adam',
                      learning_rate_init= 0.05, learning_rate='adaptive', max_iter=100,
                      random_state=33)

In [46]:
#Entrenamos el modelo lineal
model1c.fit(X_train_red, y_train)


LinearRegression(n_jobs=4)

In [47]:
y_pred_1c= model1c.predict(X_test_red)
r2_m1c = model1c.score(X_test_red, y_test)
mse1c = mean_squared_error(y_test, y_pred_1c)

print('Linear Regression noDepDelay Validation Results: R^2:', round(r2_m1c,6), 'MSE:', round(mse1c,6) )

Linear Regression noDepDelay Validation Results: R^2: 0.017154 MSE: 2550.745076


Claramente obtenemos peor resultado en las predicciones. Menor R^2 y mayor MSE